In [94]:
import pandas as pd
from dbfread import DBF

In [95]:
df = pd.DataFrame(list(DBF('raw_data/dbfs/rhc00.dbf', encoding='latin1')))
df_raw = df.copy()
df.head()

,TPCASO,SEXO,IDADE,LOCALNAS,RACACOR,INSTRUC,CLIATEN,CLITRAT,HISTFAMC,ALCOOLIS,...,UFUH,MUUH,OCUPACAO,DTDIAGNO,DTTRIAGE,DATAPRICON,DATAINITRT,DATAOBITO,VALOR_TOT,BASDIAGSP
0,2,2,071,RJ,1,9,10,88,2,1,...,RJ,3304557,888,10/03/2002,99/99/9999,02/06/2000,88/88/8888,,1,3
1,2,1,000,99,9,9,27,88,9,9,...,RJ,3304557,9999,02/01/2002,15/11/2000,11/12/2000,88/88/8888,,1,3
2,1,2,049,RJ,1,9,20,20,2,1,...,RJ,3304557,799,21/07/2002,99/99/9999,31/05/2000,21/06/2002,,1,3
3,1,1,066,ES,1,9,32,32,1,9,...,RJ,3304557,985,27/05/2002,99/99/9999,06/06/2000,15/04/2002,,1,3
4,1,2,060,99,1,9,11,4,1,1,...,RJ,3304557,9999,21/08/2002,99/99/9999,15/10/2000,09/07/2002,,1,3


In [96]:
import os, re

def ler_arquivo_cnv(caminho_arquivo):
    mapeamento = {}
    try:
        with open(caminho_arquivo, 'r', encoding='latin1') as f:
            linhas = f.readlines()

        # 1) Tenta "seq  descricao  codigo"
        achou_algo = False
        for linha in linhas:
            raw = linha.rstrip("\n")
            s = raw.strip()
            if not s or s.startswith(";"):
                continue
            m = re.match(r'^\s*(\d+)\s+(.+?)\s+([A-Za-z0-9\-]+)\s*$', raw)
            if m:
                descricao = m.group(2).strip()
                codigo = re.sub(r'[^A-Za-z0-9]', '', m.group(3))
                if codigo:
                    mapeamento[codigo] = descricao
                    achou_algo = True

        if achou_algo:
            return mapeamento

        # 2) Tenta “multi-código” (ex.: ESTADIAM)
        for linha in linhas:
            s = linha.strip()
            if not s or s.startswith(";"):
                continue

            partes = re.split(r'\s{2,}', s)
            if len(partes) < 2:
                continue
            descricao = partes[1].strip()

            if len(partes) >= 3:
                codigos = partes[2].split(",")
            else:
                continue

            for c in codigos:
                c = re.sub(r'[^A-Za-z0-9]', '', c.strip())
                if c:
                    mapeamento[c] = descricao

    except Exception as e:
        print(f"Erro ao ler arquivo {caminho_arquivo}: {e}")

    return mapeamento




def extrair_mapeamentos_def(caminho_def):
    mapeamentos = {}
    try:
        with open(caminho_def, 'r', encoding='latin1') as arquivo:
            linhas = arquivo.readlines()

        for linha in linhas:
            linha = linha.strip()
            if linha.startswith(('T', 'S')) and '.cnv' in linha:
                partes = linha.split(',')
                if len(partes) >= 4:
                    nome_coluna = partes[1].strip()
                    arquivo_cnv = partes[3].strip()
                    nome_arquivo = os.path.basename(arquivo_cnv)
                    if nome_coluna not in mapeamentos:
                        mapeamentos[nome_coluna] = nome_arquivo
    except Exception as e:
        print(f"Erro ao ler arquivo .def: {e}")

    return mapeamentos

In [97]:
mapeamentos_colunas = extrair_mapeamentos_def('raw_data/rhcGeral.def')

traducoes = {}

print("Gerando mapeamentos para todas as colunas...")
print("=" * 50)

for coluna, arquivo_cnv in mapeamentos_colunas.items():
    filepath = 'raw_data/'+arquivo_cnv
    if os.path.exists(filepath):
        mapeamento = ler_arquivo_cnv(filepath)
        if mapeamento:
            traducoes[coluna] = mapeamento
    else:
        print(f"Aviso: Arquivo {filepath} não encontrado")

print(f"Total de mapeamentos criados: {len(traducoes)}")


Gerando mapeamentos para todas as colunas...
Total de mapeamentos criados: 37


In [98]:
df.head()

,TPCASO,SEXO,IDADE,LOCALNAS,RACACOR,INSTRUC,CLIATEN,CLITRAT,HISTFAMC,ALCOOLIS,...,UFUH,MUUH,OCUPACAO,DTDIAGNO,DTTRIAGE,DATAPRICON,DATAINITRT,DATAOBITO,VALOR_TOT,BASDIAGSP
0,2,2,071,RJ,1,9,10,88,2,1,...,RJ,3304557,888,10/03/2002,99/99/9999,02/06/2000,88/88/8888,,1,3
1,2,1,000,99,9,9,27,88,9,9,...,RJ,3304557,9999,02/01/2002,15/11/2000,11/12/2000,88/88/8888,,1,3
2,1,2,049,RJ,1,9,20,20,2,1,...,RJ,3304557,799,21/07/2002,99/99/9999,31/05/2000,21/06/2002,,1,3
3,1,1,066,ES,1,9,32,32,1,9,...,RJ,3304557,985,27/05/2002,99/99/9999,06/06/2000,15/04/2002,,1,3
4,1,2,060,99,1,9,11,4,1,1,...,RJ,3304557,9999,21/08/2002,99/99/9999,15/10/2000,09/07/2002,,1,3


In [99]:
for coluna, mapa in traducoes.items():
    if coluna in df.columns:
        df[coluna] = df[coluna].astype(str).str.strip()
        mapa = {str(k).strip(): v for k, v in mapa.items()}
        df[coluna] = df[coluna].map(mapa).fillna(df[coluna])


In [100]:
df.head()

,TPCASO,SEXO,IDADE,LOCALNAS,RACACOR,INSTRUC,CLIATEN,CLITRAT,HISTFAMC,ALCOOLIS,...,UFUH,MUUH,OCUPACAO,DTDIAGNO,DTTRIAGE,DATAPRICON,DATAINITRT,DATAOBITO,VALOR_TOT,BASDIAGSP
0,Nao Analitico,Feminino,071,Rio de Janeiro,Branca,Sem informacao,DERMATOLOGIA,NAO SE APLICA,Nao,Nunca,...,Rio de Janeiro,3304557 RIO DE JANEIRO RJ,888,10/03/2002,99/99/9999,02/06/2000,88/88/8888,,1,3
1,Nao Analitico,Masculino,000,99,Sem Informacao,Sem informacao,OTORRINOLARINGOLOGIA,NAO SE APLICA,Sem Informacao,Sem Informacao,...,Rio de Janeiro,3304557 RIO DE JANEIRO RJ,9999,02/01/2002,15/11/2000,11/12/2000,88/88/8888,,1,3
2,Analitico,Feminino,049,Rio de Janeiro,Branca,Sem informacao,NEUROCIRURGIA,NEUROCIRURGIA,Nao,Nunca,...,Rio de Janeiro,3304557 RIO DE JANEIRO RJ,799,21/07/2002,99/99/9999,31/05/2000,21/06/2002,,1,3
3,Analitico,Masculino,066,Espirito Santo,Branca,Sem informacao,UROLOGIA,UROLOGIA,Sim,Sem Informacao,...,Rio de Janeiro,3304557 RIO DE JANEIRO RJ,985,27/05/2002,99/99/9999,06/06/2000,15/04/2002,,1,3
4,Analitico,Feminino,060,99,Branca,Sem informacao,ENDOCRINOLOGIA,CIRURGIA GERAL,Sim,Nunca,...,Rio de Janeiro,3304557 RIO DE JANEIRO RJ,9999,21/08/2002,99/99/9999,15/10/2000,09/07/2002,,1,3


In [101]:
def aplicar_mapa_serie(s, mapa):
    # normaliza mapa: strings sem espaços
    mapa_norm = {str(k).strip(): v for k, v in mapa.items()}

    valores_desc = set(mapa_norm.values())  # descrições
    s_str = s.astype(str).str.strip()

    def _map_one(x):
        # 0) já é descrição final?
        if x in valores_desc:
            return x
        # 1) tentativa direta
        if x in mapa_norm:
            return mapa_norm[x]
        # 2) sem zeros à esquerda (para colunas tipo CNES, MUUH etc.)
        x2 = x.lstrip('0')
        if x2 and x2 in mapa_norm:
            return mapa_norm[x2]
        # 3) maiúsculas (se houver letras)
        x3 = x.upper()
        if x3 in mapa_norm:
            return mapa_norm[x3]
        x4 = x2.upper()
        if x2 and x4 in mapa_norm:
            return mapa_norm[x4]
        # mantém original se não achou
        return x

    return s_str.map(_map_one)


In [102]:
print("\nRelatório de valores que não mapearam (deveriam mudar e não mudaram)")
print("=" * 60)

for coluna, mapa in traducoes.items():
    if coluna not in df.columns:
        continue

    mapa_norm = {str(k).strip(): v for k, v in mapa.items()}
    chaves = set(mapa_norm.keys())
    valores = set(mapa_norm.values())

    antes = df_raw[coluna].astype(str).str.strip()
    depois_esperado = aplicar_mapa_serie(antes, mapa)  # como seria o mapeamento

    # “deveria mapear” = valor do antes é uma chave (considerando lstrip('0') também)
    def e_chave(v):
        if v in chaves: return True
        v2 = v.lstrip('0')
        if v2 in chaves: return True
        if v.upper() in chaves: return True
        if v2.upper() in chaves: return True
        return False

    # filtra casos onde deveria mapear mas não mudou
    mask_deveria = antes.apply(e_chave)
    nao_mudou = (antes == depois_esperado) & mask_deveria

    if nao_mudou.any():
        exemplos = sorted(set(antes[nao_mudou]))[:10]
        print(f"\nColuna: {coluna}")
        print(f"Exemplos (até 10): {exemplos}")



Relatório de valores que não mapearam (deveriam mudar e não mudaram)

Coluna: DTPRICON
Exemplos (até 10): ['2000']

Coluna: ANOPRIDI
Exemplos (até 10): ['1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989']

Coluna: DTINITRT
Exemplos (até 10): ['1980', '1992', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006']

Coluna: ANTRI
Exemplos (até 10): ['1980', '1981', '1982', '1984', '1986', '1987', '1988', '1989', '1990', '1991']


In [103]:
# Substitui todas as ocorrências de '99' por "Não informado" apenas na coluna LOCALNAS
df['LOCALNAS'] = df['LOCALNAS'].replace('99', "Não informado")
df['PROCEDEN'] = df['PROCEDEN'].replace('9999999', "9999999 SEM INFORMACAO   OUTRO")
df['ESTADRES'] = df['ESTADRES'].replace('99', "OUTRO")
head = df.head()
head.to_csv('head_rhc00_mapeado.csv', index=False)